# Parse preprint full-texts

Contents:
1. Introduction
2. 

Check if downloaded .tex files contain fulltext. Some are just supplementary files for the submission.  If they contain \documentclass. 
Each submission folder should contain one \documentclass file. 

In [1]:
import os, sys, re, requests, subprocess, glob, multiprocessing, time, pathlib

Each submission folder may contain 1 or more .tex files. We need to identify which .tex file is the full-text preprint.

In [2]:
def get_preprints():
    base_path = 'latex'
    submissions = 0
    tex_count = 0
    empties = []
    preprints = []
    missed = []

    # Walk through tars
    for idx, tar_folder in enumerate(os.listdir(base_path)):
        tar_path = base_path + '/' + tar_folder
        if not os.path.isdir(tar_path):
            continue
        submission_dirs = os.listdir(tar_path)
        submissions += len(submission_dirs)
        # Walk through each submission
        for submission in submission_dirs:
            submission_path = tar_path + '/' + submission
            if not os.path.isdir(submission_path):
                submissions -= 1
                continue
            arxiv_id = os.path.basename(submission_path)
            texs = glob.glob(submission_path + '/**/*.tex', recursive=True)
            tex_count += len(texs)
            
            # If submission is empty, note & skip
            if len(texs) == 0:
                empties.append(arxiv_id)
                continue
            # Otherwise get the preprint
            else:
                preprint_path = getPreprint(submission_path, texs)
                if preprint_path:
                    preprints.append(preprint_path)
                else:
                    missed.append(arxiv_id)
                    
    print('Submissions: ' + str(submissions))
    print('Preprints: ' + str(len(preprints)))
    print('Non-empty submissions missing preprints: ' + str(len(missed)))
    print('Empty submissions: ' + str(len(empties)))
    print('Texs: ' + str(tex_count))
    print(str(missed))
    
    return preprints

def getPreprint(submission_path, texs):
    preprint = None
    
    # If submission contains only one file, this is the preprint
    if len(texs) == 1:
        preprint = texs[0]
    # If submission contains ms.tex or main.tex, this is the preprint
    elif 'ms.tex' in texs:
        preprint = submission_path + '/' + 'ms.tex'
    elif 'main.tex' in texs:
        preprint = submission_path + '/' + 'main.tex'
    # Otherwise, iterate through each .tex looking for \documentclass or \documentstyle
    else: 
        for tex_path in texs: 
            with open(tex_path, 'rb') as f: 
                data = f.readlines()
                r = re.compile(b'(.*\\\\documentclass.*)|(.*\\\\documentstyle.*)')
                if len(list(filter(r.match, data))) > 0:
                    preprint = tex_path
                    break
    
    return preprint

In [3]:
preprints = get_preprints()

Submissions: 89908
Preprints: 89630
Non-empty submissions missing preprints: 7
Empty submissions: 271
Texs: 125484
['1105.1087', '1211.4277', '1304.7762', '1308.6483', '1409.3422', '1606.06791', '1607.01189']


In [4]:
len(preprints)

89630

arxiv-vanity is unable to render these 7 papers as well. I'll just leave them for now.

Convert each preprint via multiprocessing.

Before you can begin multiprocessing, you need to pick which sections of code to multiprocess. These sections of code must meet the following criteria:

1. Must not be reliant on previous outcomes. True.

2. Does not need to be executed in a particular order. True.

3. Does not return anything that would need to be accessed later in the code. True.

In [ ]:
def convert_to_xml(tex_path):
    '''
    Converts TEX file to XML.
    '''
    
    print('--> Conversion beginning for {} to XML'.format(tex_path))
    # Convert given .tex to XML
    filename, file_extension = os.path.splitext(os.path.basename(tex_path))
    xml_path = 'xml/' + pathlib.Path('latex/arXiv_src_1111_009/1111.7015/DA+DAH-archive.tex').parts[2] + '.xml'
    
    if os.path.exists(xml_path):
        print(os.path.basename(tex_path) + ' has already been converted.')
    elif file_extension == '.tex':
        subprocess.call(['latexml', '--dest=' + xml_path, tex_path])
        print('--> Conversion complete for {} to XML'.format(tex_path))

def start_conversion():
    starttime = time.time()
    pool = multiprocessing.Pool(processes=4)
    # Specify a timeout so that this pool can be interrupted
    pool.map_async(convert_to_xml, preprints).get(9999999)
    pool.close()
    print('That took {} seconds'.format(time.time() - starttime))

start_conversion()

--> Conversion beginning for latex/arXiv_src_1008_005/1008.3737/stats_ms4_pdf.tex to XML
stats_ms4_pdf.tex has already been converted.
Lundqvist_pwn0540_corrected.tex has already been converted.
Pcomb.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3740/nstarB-f2.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3779/preprint.tex to XML
nstarB-f2.tex has already been converted.
preprint.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3750/Lundqvist_pwn0540_corrected.tex to XML
--> Conversion beginning for latex/arXiv_src_1107_001/1107.0722/manuscrip3.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3780/ms.tex to XML
ms.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1111_009/1111.7015/DA+DAH-archive.tex to XML
manuscrip3.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3764/ms.tex to XML
ms

--> Conversion beginning for latex/arXiv_src_1101_005/1101.3865/15991.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3874/ms.tex to XML
mnhst1.tex has already been converted.
15991.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1111_009/1111.7084/FGRB.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3866/Xe100_EMbg_v3_pdf.tex to XML
--> Conversion beginning for latex/arXiv_src_1107_001/1107.0775/radio.tex to XML
ms.tex has already been converted.
FGRB.tex has already been converted.
radio.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3875/pipino.tex to XML
--> Conversion beginning for latex/arXiv_src_1111_009/1111.7092/1111.7092.tex to XML
Xe100_EMbg_v3_pdf.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1107_001/1107.0792/moriondgoudelis.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3870/0294380.tex to XML
1111.7092

Schaffer11.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3913/1008.3913.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3976/000_ogressel.tex to XML
1008.3913.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1111_009/1111.7257/1111.7257.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3914/reionization_topology_100822.tex to XML
000_ogressel.tex has already been converted.
1111.7257.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3982/ms.tex to XML
--> Conversion beginning for latex/arXiv_src_1111_009/1111.7264/rnus_dec24.tex to XML
ms.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3984/SFS.tex to XML
rnus_dec24.tex has already been converted.
reionization_topology_100822.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1111_009/1111.7288/1111.7288.tex to XML

--> Conversion beginning for latex/arXiv_src_1107_002/1107.0974/Jimenez-Noelia.tex to XML
v13.tex has already been converted.
Jimenez-Noelia.tex has already been converted.
Mambrini.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1107_002/1107.0977/stripe_arxiv.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3999/ElectronAnalysisPaper.tex to XML
stripe_arxiv.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1112_001/1112.0012/smolec1.tex to XML
--> Conversion beginning for latex/arXiv_src_1107_002/1107.0979/ms.tex to XML
smolec1.tex has already been converted.
ms.tex has already been converted.
ElectronAnalysisPaper.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1107_002/1107.0982/ms.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_005/1008.4004/ms.tex to XML
ms.tex has already been converted.
ms.tex has already been converted.
--> Conversion beginning for latex/arXiv

--> Conversion beginning for latex/arXiv_src_1008_005/1008.4103/dm_review_rev2.tex to XML
hydro.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1112_001/1112.0068/valencia.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_005/1101.4044/ms.tex to XML
dm_review_rev2.tex has already been converted.
ms.tex has already been converted.
valencia.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_005/1101.4048/3D-AGN-torus-vf.tex to XML
--> Conversion beginning for latex/arXiv_src_1112_001/1112.0082/reply.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_005/1008.4112/manuscript_dobke_arxiv.tex to XML
3D-AGN-torus-vf.tex has already been converted.
reply.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1112_001/1112.0084/gravwave.tex to XML
manuscript_dobke_arxiv.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_005/1101.4050/turbulence.tex to XML
gravwa

...

ms.tex has already been converted.
Borkova.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1112_001/1112.0593/1112.0593.tex to XML
1008.4567.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_005/1008.4573/ifadeep.tex to XML
ifadeep.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_005/1008.4576/akari-mdwarfs-astroph.tex to XML
akari-mdwarfs-astroph.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_005/1008.4578/WBMsmallN.tex to XML
WBMsmallN.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_005/1008.4581/paper.tex to XML
paper.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_006/1101.4854/15478.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_005/1008.4582/hr8799_astroph.tex to XML
hr8799_astroph.tex has already been converted.
1112.0593.tex has already been converted.
--> Conversion beg

aipcheck.tex has already been converted.
paper_rev.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1107_003/1107.1798/sz_gong_mdi_final_review.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_006/1101.4881/aas_macros.tex to XML
aas_macros.tex has already been converted.
sz_gong_mdi_final_review.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_006/1101.4890/GravityStatistics20010523.tex to XML
--> Conversion beginning for latex/arXiv_src_1107_003/1107.1801/acc.tex to XML
GravityStatistics20010523.tex has already been converted.
acc.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_006/1101.4891/ms.tex to XML
--> Conversion beginning for latex/arXiv_src_1107_003/1107.1804/dust0617.tex to XML
dust0617.tex has already been converted.
ms.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1107_003/1107.1812/ms.tex to XML
--> Conversion beginning for latex/arXiv_

vissani.tex has already been converted.
Jeans_Instability-1.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_005/1008.4727/ms.tex to XML
--> Conversion beginning for latex/arXiv_src_1112_001/1112.0777/ms.tex to XML
ms.tex has already been converted.
ms.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1112_001/1112.0782/manuscript.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_005/1008.4730/ms.tex to XML
ms.tex has already been converted.
manuscript.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_006/1008.4741/slane.tex to XML
--> Conversion beginning for latex/arXiv_src_1112_001/1112.0786/llh_skymap.tex to XML
llh_skymap.tex has already been converted.
slane.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_006/1008.4748/gcbh_v6.tex to XML
--> Conversion beginning for latex/arXiv_src_1112_001/1112.0821/gr-qc-submission-version.tex to XML
gcbh_v6.t

brane-gw-ver13.tex has already been converted.
pagb_oh231_mnras.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_006/1101.5000/jspol0110.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_006/1008.4867/MgRadCoeff.tex to XML
--> Conversion beginning for latex/arXiv_src_1112_002/1112.1031/main_revise.tex to XML
jspol0110.tex has already been converted.
main_revise.tex has already been converted.
MgRadCoeff.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_006/1101.5004/Klochkova.tex to XML
--> Conversion beginning for latex/arXiv_src_1112_002/1112.1033/fermisymp_debbijoy.tex to XML
Klochkova.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1008_006/1008.4882/paperCesar-resub2.tex to XML
fermisymp_debbijoy.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_006/1101.5005/paper.tex to XML
paperCesar-resub2.tex has already been converted.
--> Conversion beginn

--> Conversion beginning for latex/arXiv_src_1101_006/1101.5149/helical_arxivII.tex to XML
visser280.tex has already been converted.
final_ec1.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1107_003/1107.2081/ms.tex to XML
helical_arxivII.tex has already been converted.


NOTE:
If I use `pool.map(convert_to_xml, preprints)`, I cannot kill this. So this is why I used `pool.map_async()` with a timeout specified. 
https://stackoverflow.com/questions/1408356/keyboard-interrupts-with-pythons-multiprocessing-pool

If I ever use pool.map accidentally - this is how to kill the stupid Medusa-spawning workers: https://stackoverflow.com/questions/25415104/kill-python-multiprocessing-pool


In [10]:
from pathlib import Path
Path('latex/arXiv_src_1111_009/1111.7015/DA+DAH-archive.tex').parts

('latex', 'arXiv_src_1111_009', '1111.7015', 'DA+DAH-archive.tex')

In [2]:
os.path.exists('latex/arXiv_src_1101_005/1101.3779/preprint.tex')

True

Attempt using more cores:

It is faster to use all 8 cores. So I will do the XML conversion using all 8 cores. 

First I need to confirm the main file in each repository. 
- If it doesn't contain a .bbl file, I need to add it to the bbl_lack folder. Later. Set aside and skip.
- If it doesn't contain a file, I need to retrieve it again. Later. Set aside and skip. 

I will look at each submission folder, check xml to see if a file exists with its name. If not, I will go into the submission folder to check each file if it contains \\documentclass. If it does, grab it and convert it. Break out of loop. 

Now call the functions, with conversion process.

I have 8 cores: https://superuser.com/questions/1101311/how-many-cores-does-my-mac-have/1101314#1101314
4 physical and 4 logical. 

In [165]:
def guess_extension_from_headers(h):
    """
    Given headers from an ArXiV e-print response, try and guess what the file
    extension should be.
    Based on: https://arxiv.org/help/mimetypes
    """
    if h.get('content-type') == 'application/pdf':
        return '.pdf'
    if h.get('content-encoding') == 'x-gzip' and h.get('content-type') == 'application/postscript':
        return '.ps.gz'
    if h.get('content-encoding') == 'x-gzip' and h.get('content-type') == 'application/x-eprint-tar':
        return '.tar.gz'
    # content-encoding is x-gzip but this appears to normally be a lie - it's
    # just plain text
    if h.get('content-type') == 'application/x-eprint':
        return '.tex'
    if h.get('content-encoding') == 'x-gzip' and h.get('content-type') == 'application/x-dvi':
        return '.dvi.gz'
    return None

def arxiv_id_to_source_url(arxiv_id):
    # This URL is normally a tarball, but sometimes something else.
    # ArXiV provides a /src/ URL which always serves up a tarball,
    # but if we used this, we'd have to untar the file to figure out
    # whether it's renderable or not. By using the /e-print/ endpoint
    # we can figure out straight away whether we should bother rendering
    # it or not.
    # https://arxiv.org/help/mimetypes has more info
    return 'https://arxiv.org/e-print/' + arxiv_id

def download_source_file(arxiv_id):
    """
    Download the LaTeX source of this paper and returns as ContentFile.
    """
    source_url = arxiv_id_to_source_url(arxiv_id)
    res = requests.get(source_url)
    res.raise_for_status()
    extension = guess_extension_from_headers(res.headers)
    if not extension:
        raise DownloadError("Could not determine file extension from "
                            "headers: Content-Type: {}; "
                            "Content-Encoding: {}".format(
                                res.headers.get('content-type'),
                                res.headers.get('content-encoding')))
    with open(arxiv_id + extension, 'wb+') as f:
        f.write(res.content)
        print('Created ' + arxiv_id + extension)

download_source_file('1010.3382')

Created 1010.3382.tar.gz
